In [20]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file

openai.api_key = os.getenv('OPEN_API_KEY')
openai.api_base = os.getenv('OPEN_API_BASE')
print(openai.api_key)
print(openai.api_base)

sk-FmI2uJ9bwBMGaGe4K8azA7Lmjc22mTGhA2JVTnz8dSAxiHZs
https://api.chatanywhere.cn


In [21]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,  # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,  # this is the degree of randomness of the model's output
    )
    #     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [22]:
print(get_completion(prompt="hello"))

Hello! How can I assist you today?


In [24]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role': 'user', 'content': f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role': 'assistant', 'content': f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)

In [25]:
import panel as pn  # GUI

pn.extension()

panels = []  # collect display

context = [{'role': 'system', 'content': """
你是OrderBot，一个为披萨店收集订单的自动化服务。
你首先问候顾客，然后收集订单，再询问是取货还是送货。
你等待收集整个订单，然后进行总结，并最后一次检查顾客是否想添加其他东西。
如果是送货，你就要求提供地址。
最后你收取付款。
确保澄清所有的选项、额外的东西和尺寸，以便从菜单上唯一地识别该项目。
你要用简短的、非常对话式的友好风格来回答。
该菜单包括：
意大利香肠比萨饼 12.95，10.00，7.00
奶酪比萨 10.95, 9.25, 6.50
茄子比萨11.95, 9.75, 6.75
薯条 4.50, 3.50
希腊沙拉 7.25
浇头：
额外奶酪 2.00、
蘑菇 1.50
香肠 3.00
加拿大培根 3.50
AI酱 1.50
辣椒 1.00
饮料：
可乐 3.00, 2.00, 1.00
雪碧 3.00, 2.00, 1.00
瓶装水 5.00
"""}]  # accumulate messages

inp = pn.widgets.TextInput(value="你好", placeholder='在此输入文本…')
button_conversation = pn.widgets.Button(name="发送!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='在此输入文本…')
    [1] Row
        [0] Button(name='发送!')
    [2] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)

In [26]:
messages = context.copy()
messages.append(
    {'role': 'system',
     'content': '创建一个前一个食品订单的json摘要。逐项列出每个项目的价格、  字段应该是：1）比萨饼，包括尺寸 2）配料列表 3）饮料列表，包括尺寸 4）配菜列表，包括尺寸 5）总价 '},
)
#The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},

response = get_completion_from_messages(messages, temperature=0)
print(response)

以下是前一个订单的JSON摘要：

{
  "pizza": {
    "type": "意大利香肠比萨饼",
    "size": "小号",
    "toppings": ["额外奶酪", "蘑菇"],
    "price": 14.95
  },
  "drinks": [
    {
      "type": "可乐",
      "size": "小号",
      "price": 1.00
    },
    {
      "type": "雪碧",
      "size": "小号",
      "price": 1.00
    }
  ],
  "sides": [
    {
      "type": "薯条",
      "size": "小号",
      "price": 3.50
    }
  ],
  "total_price": 20.45
}
